# Population density

## Download and unzip data

Source: https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/population-distribution-demography/geostat

In [1]:
from urllib import request
from os import path
from zipfile import ZipFile

data_dir = 'data'
url = 'http://ec.europa.eu/eurostat/cache/GISCO/geodatafiles/GEOSTAT-grid-POP-1K-2011-V2-0-1.zip'
zip_dst = path.join(data_dir, 'compressed', 'popdensity.zip')
ds_path = 'Version 2_0_1/GEOSTAT_grid_POP_1K_2011_V2_0_1.csv'
csv_dst = path.join(data_dir, 'popdensity_raw.csv')
if not path.exists(zip_dst):
    res = request.urlopen(url)
    with open(zip_dst, 'wb') as zf:
        zf.write(res.read())
if not path.exists(csv_dst):
    with ZipFile(zip_dst).open(ds_path, 'r') as ds:
        with open(csv_dst, 'wb') as dst:
            dst.write(ds.read())
print('Done!')

Done!


## Transform raw data

The format of the source data is ETRS89/LAEA in 1km squares, where the data point is the bottom left corner of the square. Note that the points are transformed from km offset to m offset

In [2]:
from os import path
import pandas as pd
import numpy as np

data_dir = 'data'
src_name = path.join(data_dir, 'popdensity_raw.csv')
dst_name = path.join(data_dir, 'popdensity.csv')

if not path.exists(dst_name):
    df = pd.read_csv(src_name, low_memory=False)
    df.drop(['CNTR_CODE','METHD_CL','YEAR','DATA_SRC','TOT_P_CON_DT'], axis='columns', inplace=True)
    pop, N, E = (np.empty(len(df.index), np.int32) for _ in range(3))
    for i, row in df.iterrows():
        s = row['GRD_ID']
        n = np.int32(s[-9:-5]) * 1000 # km to m
        e = np.int32(s[-4:]) * 1000
        pop[i] = np.int32(row['TOT_P'])
        N[i] = n
        E[i] = e
    res = pd.DataFrame({
        'pop': pop,
        'northing': N,
        'easting': E
    })
    res.to_csv(dst_name, index=False)
with open(dst_name) as f:
    print(*[next(f) for x in range(5)])

pop,northing,easting
 8,2689000,4337000
 7,2689000,4341000
 3,2690000,4341000
 3,2691000,4340000



## Mapping the data between formats

`to_latlon` and `from_latlon` can be used to map between the source format and latitude/longitude.

In [3]:
from pyproj import CRS, Transformer

crs_etrs = CRS.from_string('EPSG:3035')
crs_latlon = CRS.from_string('EPSG:4326')
transformer_to_latlon = Transformer.from_crs(crs_etrs, crs_latlon)
transformer_from_latlon = Transformer.from_crs(crs_latlon, crs_etrs)

def to_latlon(north, east): return transformer_to_latlon.transform(north, east)
def from_latlon(lat, lon): return transformer_from_latlon.transform(lat, lon)

to_latlon(*from_latlon(5,5))

(4.99999999456415, 5.000000000000003)